<a href="https://colab.research.google.com/github/ethan-huffington/Ethan-Nikolai/blob/master/Faiss_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##<font color='moccasin'>`NB SETUP`</font>

In [17]:
# Text & Image embeddings library
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 4.7 MB 37.6 MB/s 
     |████████████████████████████████| 1.3 MB 37.8 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 596 kB 37.3 MB/s 
     |████████████████████████████████| 6.6 MB 34.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=512b3acca5b0a33432af2afc41b882047b06e4503e775e1d89a567cf803fa7e6
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd
# import sentence_transformers

##<font color='moccasin'>`IMPORT SENTENCE EMBEDDINGS`</font>

Sentence Columns:
- Pair ID
- Sentence A
- Sentence B
- Relatedness Score
- 'Entailment Judgement'

In [13]:
# downloading (through 'requests' library) I think a giant text file
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

# Idk what this is doing
text = res.text

#Looks lilke a giant text file separated by '\t' for the 5 columns
#This is indexing by character it looks like
text[:200]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in a yard and an old man is standing in the background\tA group of boys in a yard is playing and a man i'

####<font color='salmon'>`Make Pandas Dataframe`

In [12]:
data.shape

(4500, 5)

In [10]:
# Using pandas parser to index by row/column values
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [11]:
# take the column 'Sentence_A' values and append them to a list
sentences = data['sentence_A'].tolist()
# Display 0-5 sentences
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [14]:
# take the column 'Sentence_A' values and append them to a list
sentence_b = data['sentence_B'].tolist()

# Possibly appending or zipping Sentences A + B
sentences.extend(sentence_b)

# Final length should be x2 the original lenght shape (4500) if the two lists were being appended
len(set(sentences))

4802

##<font color='moccasin'>`Create Embeddings w/ 'SENTENCE TRANSFORMERS'`</font>

<font color = 'pink'> Do not forget that there is a ton of complexity going on at this step - I don't know if this is a pre-trained transformer model that is merely applying embeddings to the inputted sentences or if the sentences are its training dataset, but the end result is an array of sentencenembeddings

In [18]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

(9000, 768)

In [19]:
sentence_embeddings.shape[0]

9000

In [22]:
d = sentence_embeddings.shape[1]

####<font color='salmon'>`SAVING EMBEDDINGS ON TEMPORARY`</font>

In [ ]:
# with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
#     np.save(fp, sentence_embeddings[0:256])

In [ ]:

# # saving data
# split = 256
# file_count = 0
# for i in range(0, sentence_embeddings.shape[0], split):
#     end = i + split
#     if end > sentence_embeddings.shape[0] + 1:
#         end = sentence_embeddings.shape[0] + 1
#     file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
#     with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
#         np.save(fp, sentence_embeddings[i:end, :])
#     print(f"embeddings_{file_count}.npy | {i} -> {end}")
#     file_count = int(file_count) + 1

##<font color='moccasin'>`BUILD INDEX - FLAT L2 (Euclidean Distance)`</font>

####<font color='salmon'>`INITIALIZE`</font>

In [24]:
# Initialize L2 Index, specifying vector dimensionality 'd': 768
index = faiss.IndexFlatL2(d)

✍️ Often, we will use indexes that require us to train them on our data before being used (if we are grouping or transforming the data in any way). IndexFlatL2 however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector xq during search. So, in this case, no training is required - which we can confirm by checking the is_trained attribute.

In [25]:
# This is just saying its 'trained' because it never needed any training - it is now ready to take embeddings
index.is_trained

True

####<font color='salmon'>`ADD VECTORS`</font>

In [26]:
# Add embeddings to index
index.add(sentence_embeddings)

In [27]:
# check total number of of embeddings (Should be one embedding for every sentence)
index.ntotal

9000

##<font color='moccasin'>`QUERYING`</font>

In [28]:
# Create Query Vector
xq = model.encode(["Someone sprints with a football"])

# Specify K num nearest neighbor return
k = 4

In [29]:
%%time

# Search for xq, return k neighbors
D, I = index.search(xq, k)  
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36

[[3201 7700 7704 7703]]
CPU times: user 15.1 ms, sys: 2.01 ms, total: 17.1 ms
Wall time: 17.2 ms


In [31]:
# f'{i}: = print the ith index of I[0]
# {sentences[i]}' = AND print the ith sentence in the list 'sentences'
# for every i in the vector return list I
[f'{i}: {sentences[i]}' for i in I[0]]

['3201: A group of football players is running in the field',
 '7700: A group of football players is running in the field',
 '7704: A group of football players is running in the field',
 '7703: A group of football players is running in the field']

In [ ]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

##<font color='moccasin'>`PARTITIONING THE INDEX`</font><font color='pink'>`(IVFF)`</font>

<font color = pink>(For making querying faster) The below will essentially cluster the n-dim space to n centroids and form voronoi partitions from the clusters.

✍️ FAISS allows us to add an additional step to optimize our search efficiency using a variety of different methods. A popular approach is to partition the index into Voronoi cells. Using this method we would take our query vector xq, identify the cell it belongs to, and then use our IndexFlatL2 to search between the query vector xq and all indexed vectors belonging to that cell. We can also include vectors from other nearby cells too.

In [32]:
# Specify number of centroids
nlist = 50

# Instantiate index, again specifiying euclidean distance in d-dimensional space
  # Now however, this is a 'quantization' step (sub-algorithm) in the entire search algorithm
quantizer = faiss.IndexFlatL2(d)

# Instantiate Voronoi Partition index, w/ required arguments
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [33]:
# Now, the index needs to train the clusterings so it should NOT say its trained and ready
index.is_trained

False

####<font color='salmon'>`INDEX TRAINING`</font>

In [34]:
# Pass index the embeddings and tell it to form n clusters
index.train(sentence_embeddings)
# Check for training status again
index.is_trained

True

####<font color='salmon'>`ADD VECTORS`</font>

In [35]:
# Add same embeddings into new index
index.add(sentence_embeddings)

# check the number of embeddings in index
index.ntotal

9000

In [36]:
# Run the same time check on the new search algorithm
%%time
D, I = index.search(xq, k)
print(I)

[[3201 7703 7704 7700]]
CPU times: user 2.21 ms, sys: 5 µs, total: 2.22 ms
Wall time: 6.24 ms


####<font color='salmon'>`nprobe`</font>

In [37]:
# Specify the number of nearest centroid voronoi-cells to search
index.nprobe = 10

In [38]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[3201 7703 7704 7700]]
CPU times: user 6.11 ms, sys: 0 ns, total: 6.11 ms
Wall time: 9.12 ms


Increasing the number of nprobe will improve the accuracy of our search, but cost time. Our earlier IndexFlatL2-only search was exhaustive (it compared every single vector) and so it identified the closest matches with a perfect accuracy. The smaller our nprobe value, the smaller scope that we search. We received perfect results (that matched our previous IndexFlatL2-only results - 7460, 10940, 3781, 5747), however, if we found that we were not getting closely matching results, we could simply bump nprobe up further - improving accuracy, but increasing time-taken too.

##<font color='moccasin'>`PRODUCT QUANTIZATION`</font>

PQ achieves this approximated distance operation by compressing the vectors themselves. This consists of a few steps.

1 - We split the every original vector into several subvectors.

2 - For each set of subvectors, we perform a clustering operation, creating many centroids for each subvector set.

3 - In our vector of subvectors, we replace each subvector with the ID of it's nearest centroid.



In [40]:
# number of centroid IDs in final compressed vectors
m = 8

# number of bits in each centroid
bits = 8 

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [41]:
# Train the sub-vector clustering
index.train(sentence_embeddings)

In [42]:
# Add vectors
index.add(sentence_embeddings)

In [ ]:
# Respecify nprode value
index.nprobe = 10

In [43]:
# Check out the time performance
%%time
D, I = index.search(xq, k)
print(I)

[[  46 7894 3390 3201]]
CPU times: user 2.94 ms, sys: 0 ns, total: 2.94 ms
Wall time: 3.04 ms


In [44]:
[f'{i}: {sentences[i]}' for i in I[0]]

['46: Two groups of people are playing football',
 '7894: A football player is running past an official carrying a football',
 '3390: A football player is running past an official carrying a football',
 '3201: A group of football players is running in the field']

<font color = 'pink'> Looks like the increased voronoi exploration has actually imporved the results in this limited case because the other returns were all the same and boring anyway